In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/t5-finetune-data-csv/t5_finetune_data.csv
/kaggle/input/10000values/10000Values.csv


In [5]:
pip install transformers datasets

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


# Fine Tuning-**google/t5-v1_1-small**256charlen

In [7]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datasets import Dataset
import pandas as pd

# Load the dataset
dataset_path = '/kaggle/input/10000values/10000Values.csv'
finetune_data = pd.read_csv(dataset_path)

# Ensure that the input and target texts are strings and fill NaN values
finetune_data['input_text'] = finetune_data['input_text'].astype(str)
finetune_data['target_text'] = finetune_data['target_text'].astype(str)

# Convert the pandas DataFrame to Hugging Face Dataset format
hf_dataset = Dataset.from_pandas(finetune_data)

# Load tokenizer and model
model_name = "google/t5-v1_1-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Tokenization function with proper input handling
def preprocess_function(examples):
    inputs = [str(ex) for ex in examples['input_text']]
    targets = [str(ex) for ex in examples['target_text']]
    
    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=256, truncation=True, padding="max_length")
    
    # Replace any padding token in the labels to -100 so it's ignored by the loss function
    labels["input_ids"] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in labels_seq]
        for labels_seq in labels["input_ids"]
    ]
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize the dataset
tokenized_dataset = hf_dataset.map(preprocess_function, batched=True)

# Training arguments (set according to your GPU/CPU capacity)
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./t5_finetune_results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    fp16=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_steps=500,
    logging_dir="./logs",
    save_safetensors=False  # Disable safetensors to avoid serialization issue
)


# Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
)

# Fine-tune the model
trainer.train()


tokenizer_config.json:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,2.152200,0.789282
2,0.915200,0.560061
3,0.797800,0.497412


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=3753, training_loss=1.8820424251419177, metrics={'train_runtime': 1558.4137, 'train_samples_per_second': 19.26, 'train_steps_per_second': 2.408, 'total_flos': 2789751749345280.0, 'train_loss': 1.8820424251419177, 'epoch': 3.0})

# Saving the model

In [8]:
trainer.save_model("./t5-google-v1_1-finetuned-model")
tokenizer.save_pretrained("./t5-google-v1_1-finetuned-tokenizer")

('./t5-google-v1_1-finetuned-tokenizer/tokenizer_config.json',
 './t5-google-v1_1-finetuned-tokenizer/special_tokens_map.json',
 './t5-google-v1_1-finetuned-tokenizer/spiece.model',
 './t5-google-v1_1-finetuned-tokenizer/added_tokens.json')

# Evaluating

In [1]:
##now testing it by just giving a course name
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the fine-tuned model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("./t5-google-v1_1-finetuned-model")
tokenizer = T5Tokenizer.from_pretrained("./t5-google-v1_1-finetuned-tokenizer")

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Function to generate roadmap based on course name
def generate_roadmap(course_name):
    # Create a structured input prompt for the model
    input_text = f"Course: {course_name}, Level: Beginner, Generate a complete learning roadmap."
    
    # Tokenize the input text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    
    # Move input tensors to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    # Generate a learning path
    outputs = model.generate(inputs['input_ids'], max_length=150, num_return_sequences=1)
    
    # Decode and return the generated output
    generated_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_output

# Test with an example course input
course_name =input()
roadmap = generate_roadmap(course_name)
print(roadmap)


OSError: Incorrect path_or_model_id: './t5-google-v1_1-finetuned-model'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [14]:
from IPython.display import FileLink

# Specify the file path
file_path = '/kaggle/working/t5-google-v1_1-finetuned-model/pytorch_model.bin'

# Generate a download link
FileLink(file_path)


/kaggle/working/t5-google-v1_1-finetuned-model/pytorch_model.bin